In [2]:
import pdfplumber
import pandas as pd
import re

In [3]:
import sys
print(sys.executable)


C:\Users\PC\miniconda3\envs\ml\python.exe


In [7]:
from pathlib import Path

base_dir = Path(r"D:\Documents\fish")

pdf_files = list(base_dir.rglob("*.pdf"))

print(f"Found {len(pdf_files)} PDF files")
for pdf in pdf_files[:10]:   # preview first 10
    print(pdf)


Found 139 PDF files
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\1.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\10.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\11.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\12.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\13.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\14.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\2.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\3.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\4.pdf
D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD GARU\GANESH\5.pdf


In [11]:
type(pdf_files)

list

In [9]:
def extract_pdf_data(pdf_path):
    records = []

    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]  # assuming all data on first page
        table = page.extract_table()
        text = page.extract_text()

        if not text or not table:
            return records

        # ---- Extract metadata (SAFE regex) ----
        metadata = {}
        try:
            metadata["farmer_name"] = re.search(
                r"Farmer's Name:\s*(.*?)\s*S\.No\.:", text
            ).group(1)
            metadata["area"] = re.search(
                r"Area:\s*(.*?)\s*No\.Of Samples:", text
            ).group(1)
            metadata["sample_no"] = re.search(
                r"S\.No\.\:\s*(\S+)", text
            ).group(1)
            metadata["date_time"] = re.search(
                r"Date:\s*(.*)", text
            ).group(1)
            metadata["no_of_samples"] = re.search(
                r"No\.Of Samples:\s*(\d+)", text
            ).group(1)
        except AttributeError:
            print(f"⚠️ Metadata regex failed for {pdf_path.name}")
            return records

        # ---- Extract table ----
        df = pd.DataFrame(table[1:], columns=table[0])

        df.columns = (
            df.columns
              .str.strip()
              .str.lower()
              .str.replace(" ", "_")
        )

        df = df.replace(["NIL", ""], None)
        df = df.dropna(how="all")

        # ---- Merge metadata into each row ----
        for _, row in df.iterrows():
            rec = metadata.copy()
            rec.update(row.to_dict())
            rec["file_name"] = pdf_path.name
            rec["folder_path"] = str(pdf_path.parent)
            records.append(rec)

    return records


In [10]:
all_records = []

for pdf_path in pdf_files:
    print(f"Processing: {pdf_path.relative_to(base_dir)}")
    all_records.extend(extract_pdf_data(pdf_path))

final_df = pd.DataFrame(all_records)
final_df.head()


Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\1.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\10.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\11.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\12.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\13.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\14.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\2.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\3.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\4.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\5.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\6.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\7.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\8.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\GANESH\9.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\MURTHY RAJU\1.pdf
Processing: HARI PRASAD GARU\HARI PRASAD GARU\MURTHY RAJU\2.pdf
Processing: HARI PRASAD GARU\HARI PRASAD 

,farmer_name,area,sample_no,date_time,no_of_samples,s.no,pond,acre,d.o.c,ph,...,do,hardness\n(ppm),ca2+\n(ppm),mg+\n(ppm),na+\n(ppm),k+\n(ppm),h2s\n(ppm),chlorine\n(ppm),file_name,folder_path
0,MR.GANESH GARU,KONDURU,GAL/4,06-Jan-2026 09:15,1,1,E.T,4,None,8.6,...,4.5,1450,130,320,None,None,None,None,1.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
1,MR.GANESH GARU,KONDURU,GAL/16,05-Feb-2025 09:53,3,1,SHP,1,30,8.4,...,5.0,4600,460,1006,None,None,None,None,10.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
2,MR.GANESH GARU,KONDURU,GAL/16,05-Feb-2025 09:53,3,2,SHP,2,30,8.3,...,4.5,3850,390,841,None,None,None,None,10.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
3,MR.GANESH GARU,KONDURU,GAL/16,05-Feb-2025 09:53,3,3,SHP,4,65,7.9,...,5.0,2900,300,631,None,None,None,None,10.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
4,MR.GANESH GARU,ARUTEGALAPADU,GAL/13,30-Jan-2025 08:58,2,1,SHP,4,60,7.9,...,5.0,3250,330,791,None,None,None,None,11.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...


In [12]:
display(final_df)

,farmer_name,area,sample_no,date_time,no_of_samples,s.no,pond,acre,d.o.c,ph,...,do,hardness\n(ppm),ca2+\n(ppm),mg+\n(ppm),na+\n(ppm),k+\n(ppm),h2s\n(ppm),chlorine\n(ppm),file_name,folder_path
0,MR.GANESH GARU,KONDURU,GAL/4,06-Jan-2026 09:15,1,1,E.T,4,None,8.6,...,4.5,1450,130,320,None,None,None,None,1.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
1,MR.GANESH GARU,KONDURU,GAL/16,05-Feb-2025 09:53,3,1,SHP,1,30,8.4,...,5.0,4600,460,1006,None,None,None,None,10.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
2,MR.GANESH GARU,KONDURU,GAL/16,05-Feb-2025 09:53,3,2,SHP,2,30,8.3,...,4.5,3850,390,841,None,None,None,None,10.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
3,MR.GANESH GARU,KONDURU,GAL/16,05-Feb-2025 09:53,3,3,SHP,4,65,7.9,...,5.0,2900,300,631,None,None,None,None,10.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
4,MR.GANESH GARU,ARUTEGALAPADU,GAL/13,30-Jan-2025 08:58,2,1,SHP,4,60,7.9,...,5.0,3250,330,791,None,None,None,None,11.pdf,D:\Documents\fish\HARI PRASAD GARU\HARI PRASAD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,MR.VENKATA RAJU GARU,AI-BHIMAVARAM,GAL/8,20-Mar-2025 08:39,1,1,E.T,1.5,None,8.4,...,5.0,950,90,209,None,None,None,None,6.pdf,D:\Documents\fish\VENKATA RAMA RAJU GARU AI-BH...
189,MR.VENKATA RAJU GARU,AI-BHIMAVARAM,GAL/14,24-Mar-2025 08:50,1,1,E.T,8,None,8.5,...,5.0,750,70,165,None,None,None,None,7.pdf,D:\Documents\fish\VENKATA RAMA RAJU GARU AI-BH...
190,MR.VENKATA RAJU GARU,AI BHIMAVARAM,GAL/7,04-Apr-2025 08:32,2,1,SHP,1.5,35,8.3,...,5.0,2550,250,559,None,None,None,None,8.pdf,D:\Documents\fish\VENKATA RAMA RAJU GARU AI-BH...
191,MR.VENKATA RAJU GARU,AI BHIMAVARAM,GAL/7,04-Apr-2025 08:32,2,2,SHP,6,35,8.8,...,5.5,1450,140,318,None,None,None,None,8.pdf,D:\Documents\fish\VENKATA RAMA RAJU GARU AI-BH...
